# 005 TEST

# Function Call

In [ ]:
%pip install ipympl

In [ ]:
%pip install tiktoken openai pandas matplotlib plotly scikit-learn numpy python-dotenv matplotlib widget

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  


import openai
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [ ]:
import sqlite3
conn = sqlite3.connect("data/chinook.db")
print("opened db successfully")

In [ ]:
def get_table_names(conn):
    table_names = []  
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names  


def get_column_names(conn, table_name):
    column_names = []  
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names  


def get_database_info(conn):
    table_dicts = []  
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)  
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  

In [ ]:
database_schema_dict = get_database_info(conn)

database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [ ]:
database_schema_dict